In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [ ]:
# Load Data
file_path = "/Users/dylanfei/Downloads/Dylan Fei - Distillate Dataset for Forecasts (Summer Internship Dataset).xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.index.freq = 'MS'  # Set Monthly Start frequency
df.columns = df.columns.str.strip()

In [ ]:
# Target & Exogenous Variables
y = df['Distillate Demand'].copy()
exog = df[['HO1 (Heating Oil) Index', 'Agricultural Indices']].copy()

# 12-Month Train/Test Split
train = y.iloc[:-12]
test = y.iloc[-12:]
exog_train = exog.iloc[:-12]
exog_test = exog.iloc[-12:]

In [ ]:
# SARIMAX Model for Evaluation
model = SARIMAX(train, exog=exog_train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12),
                enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False, maxiter=1000)

In [ ]:
# Forecast
forecast = results.get_prediction(start=test.index[0], end=test.index[-1], exog=exog_test)
predicted_mean = forecast.predicted_mean

# Error Metrics
rmse = sqrt(mean_squared_error(test, predicted_mean))
mae = mean_absolute_error(test, predicted_mean)

In [ ]:
# Plot 12-Month Forecast vs Actual
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Training Data', color='gray', alpha=0.6)
plt.plot(test.index, test, label='Actual Demand', color='darkblue', linewidth=2)
plt.plot(test.index, predicted_mean, label='Forecasted Demand', color='lightblue', linestyle='--', linewidth=2)

metrics_text = f"RMSE: {rmse:.2f} KBD\nMAE: {mae:.2f} KBD"
plt.gca().text(0.98, 0.02, metrics_text, transform=plt.gca().transAxes,
               fontsize=10, verticalalignment='bottom', horizontalalignment='right',
               bbox=dict(boxstyle="round,pad=0.4", facecolor='white', alpha=0.7))
plt.title("Distillate Demand Forecast (12-Month Evaluation)")
plt.xlabel("Date")
plt.ylabel("KBD")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Forecasting Future Months
n_months = 3
latest_exog_values = exog.iloc[-1].values
future_months = pd.date_range(start=y.index[-1] + pd.DateOffset(months=1), periods=n_months, freq='MS')
future_exog_months = pd.DataFrame([latest_exog_values] * n_months, index=future_months, columns=exog.columns)

# Refit Full Model
final_model = SARIMAX(y, exog=exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12),
                      enforce_stationarity=False, enforce_invertibility=False)
final_results = final_model.fit(disp=False, maxiter=1000)

In [ ]:
# Forecast Next 3 Months
monthly_forecast = final_results.get_forecast(steps=n_months, exog=future_exog_months)
monthly_forecast_values = monthly_forecast.predicted_mean
conf_int = monthly_forecast.conf_int()
conf_int.index = future_months

# Prepare for Weekly Interpolation
last_value = y.iloc[-1]
last_conf_int = pd.DataFrame([[last_value, last_value]], index=[y.index[-1]], columns=conf_int.columns)
monthly_forecast_full = pd.concat([y[-1:], monthly_forecast_values])
conf_int_full = pd.concat([last_conf_int, conf_int])

In [ ]:
# Weekly Interpolation for Prompt Weeks (Fridays from July to October 2025)
weekly_range = pd.date_range(start="2025-07-04", end="2025-10-31", freq='W-FRI')
monthly_forecast_full.index = pd.to_datetime(monthly_forecast_full.index)

# Interpolate Weekly Forecast and Bounds
interpolated = monthly_forecast_full.reindex(monthly_forecast_full.index.union(weekly_range)).interpolate('time').reindex(weekly_range)
lower_interp = conf_int_full.iloc[:, 0].reindex(conf_int_full.index.union(weekly_range)).interpolate('time').reindex(weekly_range)
upper_interp = conf_int_full.iloc[:, 1].reindex(conf_int_full.index.union(weekly_range)).interpolate('time').reindex(weekly_range)

# Combine Weekly Forecast and Confidence Intervals
weekly_forecast_with_ci = pd.DataFrame({
    "Forecasted Weekly Demand (KBD)": interpolated,
    "Lower Bound (95%)": lower_interp,
    "Upper Bound (95%)": upper_interp
})

In [ ]:
# Export to Excel
output_path = "/Users/dylanfei/Downloads/Distillate_Weekly_Forecast_July_October_2025.xlsx"
weekly_forecast_with_ci.to_excel(output_path)